# Example usage of rdsolver

(c) 2017 Justin Bois. This work is licensed under a [Creative Commons Attribution License CC-BY 4.0](https://creativecommons.org/licenses/by/4.0/). All code contained herein is licensed under an [MIT license](https://opensource.org/licenses/MIT).

`rdsolver` solves the following system of PDEs on a 2D Cartesian domain with periodic boundary conditions.

\begin{align}
\partial_t c_i = D_i(\partial_x^2 + \partial_y^2)c_i + \beta_i + \gamma_i c_i + f_i(c_1, c_2, \ldots, t)\,
\end{align}

where $i$ denotes chemical species. Note that the terms $\beta_i$, $\gamma_i c_i$, and $f_i(c_1, c_2, \ldots)$ are the chemical reaction terms. They are split between linear ($\beta_i$, $\gamma_i c_i$) and nonlinear terms ($f_i$), though there can be linear terms linear in $c_j, j\ne i$ in $f_i$.

To specify the problem, the user needs to supply: 

* The physical dimension of the system, which we will call $\mathbf{L} = (L_x, L_y)$
* The number of grid points $\mathbf{n} = (n_x, n_y)$
* The desired time points $t_0, t_1, \ldots$
* The initial concentration profiles of all species, $\mathbf{c}^0(x, y)$
* The values of the parameters $D_i$, $\beta_i$, and $\gamma_i$ for each species
* The function $f_i$ and any parametric arguments that need to be passed to it.

Here, I present an example of how to use `rdsolver`. To learn more about it and installation instructions, see the [README file](https://github.com/justinbois/rdsolver#reaction-diffusion-solver).

## Necessary imports

We work with `rdsolver` in a Jupyter notebook (recommended), you need to import `rdsolver` and `bokeh`, being sure to call `bokeh.io.output_notebook()` to enable interactive plotting in the Jupyter notebook. And it's pretty much automatic to import NumPy!

In [1]:
import numpy as np

import rdsolver as rd

import bokeh.io
bokeh.io.output_notebook()

Loading BokehJS ...

## Example 1: The activator-substrate depletion model (ASDM)

For our first example, we will use the ASDM, a classic system that gives Turing patterns. The dimensionless equations are

\begin{align}
\partial_t a &= d(\partial_x^2 + \partial_y^2)a + a^2s - a \\[1em]
\partial_t s &= (\partial_x^2 + \partial_y^2)s + \mu(1 - a^2s).
\end{align}

Thus, we have

\begin{align}
D &= (d, 1) \\
\beta &= (0, \mu) \\
\gamma &= (-1, 0).
\end{align}

We start by specifying the easy stuff, the physical size of the system, the number of grid points, and the time points we want. Because this particular system does not have very sharp gradients and we are not using a very large physical space, we do not need many grid points at all. Because `rdsolver` uses spectral methods, we can have very accurate calculations even with few grid points. We will use as 32 $\times$ 32 grid here.

In [8]:
# Physical size of system
L = (10, 10)

# Number of grid points in x and y (can often be small with spec. meth.)
n = (32, 32)

# Specify times points we want
t = np.linspace(0, 250, 100)

Now, we need to define our parameters. We'll start with $D$, $\beta$, and $\gamma$, choosing $d = 0.05$ and $\mu = 1.4$.

In [9]:
d = 0.05
mu = 1.4

D = (d, 1)
beta = (0, mu)
gamma = (-1, 0)

Now, we need to define our nonlinear function $f$. This function has call signature `f(u, t, *f_args)`. The first argument, `u`, is an array containing the concentrations that can be unpacked as follows `a, s = u`.

Next, the function `f` takes the current time as an input. For the ASDM, and indeed for many R-D applications, the nonlinear chemical dynamics do not explicitly depend on time. Finally, `f_args` is a tuple containing any other arguments the function `f` needs.

The function must return an array the same shape as the input array `u` that gives the nonlinear terms in the dynamics. This is most easily accomplished using the `np.stack()` function.

In [10]:
def f(u, t, mu):
    """Nonlinear terms for ASDM"""
    a, s = u
    return np.stack((a**2 * s, -mu * a**2 * s))

We also have to specify the arguments that need to be passed to `f` as a tuple.

In [11]:
# Specify the arguments that need to be passed to f
f_args = (mu, )

Now, we need to specify the initial conditions. The initial conditions must be a three-dimensional array of shape $(n_s, n_x, n_y)$, where $n_s$ is the number of chemical species. For convenience, you can specify a homogeneous steady state and use the `rd.initial_condition()` function that will generate an initial condition that is a small perturbation about the specified steady state.

In [12]:
# Homogenous steady state in activator and substrate to perturb
uniform_conc = (1, 1)

# Generate perturbed initial condition
c0 = rd.initial_condition(uniform_conc=uniform_conc, n=n, L=L)

# Show the shape as a demonstration
c0.shape

(2, 32, 32)

Now, everything is in place. We just have to solve. We do this by calling the `rd.solve()` function. The arguments are obvious from the call below.

In [14]:
# Solve the system
c = rd.solve(c0, t, D=D, beta=beta, gamma=gamma, f=f, f_args=f_args, L=L)

# Look at the shape of the solution
c.shape

(2, 32, 32, 100)

We note that the solution is of the shape $(n_s, n_x, n_y, n_t)$, where $n_t$ is the number of time points we used. This structure is useful to know for slicing out species and time points of interest.

For plotting purposes, it is useful to interpolate the solution to have smooth concentration profiles. This is purely aesthetic; the solver will give a pixelated, but spectrally accurate, solution. We can use the `rd.viz.interpolate_concs()` function to get the interpolated concentration profiles.

In [15]:
c_interp = rd.viz.interpolate_concs(c)

Finally, we are ready to display the solution. We use the `rd.viz.display_notebook()` function that will give a picture of the concentration field with a slider for adjusting the time. By default, for multiple species problems, (like the ASDM), up to three species are shown. The cyan channel is the first, magenta the second, and yellow is the third (though it is not present for the ASDM).

In [16]:
rd.viz.display_notebook(t, c_interp)

If we like, we can look at a single species, in which case the colormap is Viridis.

In [17]:
rd.viz.display_notebook(t, c_interp[0])

## A second example: FitzHugh-Nagumo

For a second example, we consider a model of the FitzHugh-Nagumo type.

\begin{align}
\partial_t u &= d (\partial_x^2 + \partial_y^2)u - u(u+\alpha)(u-1) - v, \\[1em]
\partial_t v &= (\partial_x^2 + \partial_y^2)v + (u - v),
\end{align}

It is useful to expand out the first equation.

\begin{align}
\partial_t u &= d (\partial_x^2 + \partial_y^2)u - u^3 + (1-\alpha)u^2 + \alpha u - v.
\end{align}

I will not narrate all the steps here, but set up the problem. We will choose parameters for which no homogeneous steady state exists, and will instead choose an arbitrary initial condition.

In [ ]:
# Grid
L = (20, 20)
n = (128, 128)

# Time points
t = np.linspace(0, 250, 100)

# Parameters
d = 0.15
alpha = 0.04

# Set up D, beta, and gamma
D = (d, 1)
beta = (0, 0)
gamma = (alpha, -1)

# Define nonlinear function
def f(uv, t, alpha):
    """FitzHugh-Nagumo nonlinear terms."""
    u, v = uv
    return np.stack((-u**3 + (1-alpha)*u**2 - v, u))

f_args = (alpha,)

# Starting condition
uniform_conc = (1, 1)
c0 = rd.initial_condition(uniform_conc=uniform_conc, n=n, L=L)

# Solve
c = rd.solve(c0, t, D=D, beta=beta, gamma=gamma, f=f, f_args=f_args, L=L, allow_negative=True)

Now, we can interpolate and visualize.

In [32]:
# Interpolate
c_interp = rd.viz.interpolate_concs(c)

# Visualize
rd.viz.display_notebook(t, c_interp)

## Min system

The dynamical equations are

\begin{align}
\partial_t c_\mathrm{D} &= D_\mathrm{D}(\partial_x^2 + \partial_y^2)c_\mathrm{D} + \omega_\mathrm{de} c_\mathrm{de} - \omega_\mathrm{dD}c_\mathrm{d} c_\mathrm{D} - \omega_\mathrm{D}  c_\mathrm{D} \\[0.5em]
\partial_t c_E &= D_\mathrm{E}(\partial_x^2 + \partial_y^2)c_\mathrm{E} + \omega_\mathrm{de} c_\mathrm{de} - \omega_\mathrm{E} c_\mathrm{d} c_\mathrm{E} - \omega_\mathrm{eE} c_\mathrm{d} c_\mathrm{E} c_\mathrm{de}^2 \\[0.5em]
\partial_t c_\mathrm{d} &= D_\mathrm{d}(\partial_x^2 + \partial_y^2)c_\mathrm{d} + \omega_\mathrm{D} c_\mathrm{D} + \omega_\mathrm{dD}c_\mathrm{d}c_\mathrm{D} - \omega_\mathrm{E}c_\mathrm{d}c_\mathrm{E} - \omega_\mathrm{eE}c_\mathrm{d}c_\mathrm{E}c_\mathrm{de}^2 \\[0.5em]
\partial_t c_\mathrm{de} &= D_\mathrm{de}(\partial_x^2 + \partial_y^2)c_\mathrm{de} + \omega_\mathrm{E}c_\mathrm{d}c_\mathrm{E} + \omega_\mathrm{eE}c_\mathrm{d}c_\mathrm{E}c_\mathrm{de}^2 - \omega_\mathrm{de}c_\mathrm{de}.
\end{align}